# Modeling air pollution

Let us try to predict Beijing's air pollution, especially [PM2.5](https://en.wikipedia.org/wiki/Particulates) values in advance!


Inspiration comes from [here](https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/).

## Dataset

[Beijing PM2.5 Data Data Set](https://archive.ics.uci.edu/ml/datasets/Beijing+PM2.5+Data)

### Columns of the dataset:

**No:** row number

**year:** year of data in this row

**month:** month of data in this row

**day:** day of data in this row

**hour:** hour of data in this row

**pm2.5:** PM2.5 concentration

**DEWP:** Dew Point

**TEMP:** Temperature

**PRES:** Pressure

**cbwd:** Combined wind direction

**Iws:** Cumulated wind speed

**Is:** Cumulated hours of snow

**Ir:** Cumulated hours of rain

In [ ]:
!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/pollution.csv

In [ ]:
!pip install seglearn

In [ ]:
import pandas as pd 
import csv
import numpy as np 
import matplotlib.pyplot as plt

#import warnings
#warnings.filterwarnings("ignore")


In [ ]:
def sniff_format(location):
    with open(location, newline='') as csvfile:
        sniffer = csv.Sniffer()
        sample = csvfile.read(1024)
        dialect = sniffer.sniff(sample)
        header = sniffer.has_header(sample)
        if header:
            header=0
        else:
            header=None

    return {"dialect":dialect, "header":header}

def describe_full(df):
    #pd.options.display.float_format = '{:.2f}'.format
    dtypes_description=pd.DataFrame(dict(df.dtypes),["dtypes"])
    na_description = pd.DataFrame(dict(df.isna().sum()),["NA-s"])
    na_percent = ((pd.DataFrame(dict(df.isna().sum()),["NA%"])/len(df))*100).round(decimals=2)
    description = df.describe(include='all')
    full_description = dtypes_description.append(na_description).append(na_percent).append(description).replace(np.nan, '', regex=True)

    mask = full_description.loc["freq",:]==1
    full_description.at[["top"],mask.index[mask]]=""
    #TODO: scientific notation - could be nicer
    
    return full_description

In [ ]:
csv_format = sniff_format("pollution.csv")

df = pd.read_csv("pollution.csv",header=csv_format["header"],dialect=csv_format["dialect"])

#There is a warning that would be worth investigationg, but for now, let's ignore it
import warnings
warnings.filterwarnings("ignore")

describe_full(df)

In [ ]:
#Safety assertions to ensure, that No is as it seems an index column sorted, series
pd.testing.assert_series_equal(df.No,df.No.sort_values())
np.testing.assert_array_equal((df.No-df.No[0]).values, np.arange(0,df.shape[0],1))
#Don't really need these if we decide to go for date based indexing

In [ ]:
df.drop("No",axis=1, inplace=True)
df["date"]= pd.to_datetime(df['year'].astype(str)+'-'+df['month'].astype(str)+"-"+df["day"].astype(str)+"T"+df["hour"].astype(str).apply(lambda x: x.zfill(2)+":00"))
df.set_index(df.date, inplace=True)
df.drop("date", axis=1, inplace=True)
df.head(10)

## Encoding day of week

We explicitly encode the day of week, since we assume that weekends and workdays behave differently.

In [ ]:
df["dayofweek"]=df.index.dayofweek+1

# Decision about NaN-s

In [ ]:
fig = plt.gcf()
fig.set_size_inches(10,7)
df["pm2.5"].isnull().astype(float).plot()

In [ ]:
def isnullsum(df):
    return df.isnull().sum()
print("1-----")
print("% NaN datapoints per year:",(df.groupby(df.index.year)["pm2.5"].apply(isnullsum)/df.groupby(df.index.year)["pm2.5"].count())*100.0)

print("2--------------------------------")


print("% NaN datapoints per month:",(df.groupby(df.index.month)["pm2.5"].apply(isnullsum)/df.groupby(df.index.month)["pm2.5"].count())*100.0)

print("3--------------------------------")

print("% NaN datapoints per dayofweek:",(df.groupby(df.index.dayofweek)["pm2.5"].apply(isnullsum)/df.groupby(df.index.dayofweek)["pm2.5"].count())*100.0)

print("4--------------------------------")
print("% NaN datapoints per hour:",(df.groupby(df.index.hour)["pm2.5"].apply(isnullsum)/df.groupby(df.index.hour)["pm2.5"].count())*100.0)


After examining the NaN values in pm2.5, we see no obvious temporal pattern. This is cause for worry, since by simply dropping the rows with NaN values, we can destroy the temporal coherence of the data, hence **data imputation is desirable.**

The autocorrelation charts below imply, that it is not unreasonable to take the previous value to fill NaN-s (high autocorrelation with the previous timestep).

In [ ]:

df.fillna(method='ffill', inplace=True)

print(df.isnull().sum())

df.dropna(inplace=True)

print(df.isnull().sum())


# Examining autocorrelations

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf

#columns = [] #use this for speedup
columns = ["pm2.5","DEWP","TEMP","PRES","Iws","Is","Ir"]

for col in columns:
    plt.figure()
    plot_pacf(df[col].dropna(), lags=200, zero=False)
    
plt.show()


## What do we see?

Well, the fact, that we don't see.

Or more precisely: smog (and weather) is slow to move, it is extremely strongly autocorrelated with itself one-two hours before, so in order to at least be able to see some autocorrelation structure beyond this, we need to filter out the first some hours from our autocorrelation analysis.

(By the way, that's why we don't stick to the prediction of the next hour as in the original "inspiration" blogpost. Would not be too relevant...)

In [ ]:
from statsmodels.graphics.tsaplots import _prepare_data_corr_plot, _plot_corr
import statsmodels.graphics.utils as utils
from statsmodels.tsa.stattools import pacf

def plot_pacf_drop(x, ax=None, lags=None, alpha=.05, method='ywunbiased',
              use_vlines=True, title='Partial Autocorrelation', zero=True,
              vlines_kwargs=None, drop_no=0, **kwargs):
    
    lags_orig=lags
    fig, ax = utils.create_mpl_ax(ax)
    vlines_kwargs = {} if vlines_kwargs is None else vlines_kwargs
    lags, nlags, irregular = _prepare_data_corr_plot(x, lags, zero)
    confint = None
    if alpha is None:
        acf_x = pacf(x, nlags=nlags, alpha=alpha, method=method)
    else:
        acf_x, confint = pacf(x, nlags=nlags, alpha=alpha, method=method)

    if drop_no:
        acf_x = acf_x[drop_no+1:]
        confint = confint[drop_no+1:]
        lags, nlags, irregular = _prepare_data_corr_plot(x, lags_orig-drop_no, zero)
        
    _plot_corr(ax, title, acf_x, confint, lags, False, use_vlines,
               vlines_kwargs, **kwargs)

    return fig

In [ ]:
import matplotlib.pyplot as plt

#columns = [] #use this for speedup
columns = ["pm2.5","DEWP","TEMP","PRES","Iws","Is","Ir"]

for col in columns:
    plt.figure()
    plot_pacf_drop(df[col].dropna(), lags=200, drop_no=3, zero=False)
    
plt.show()


Studying even the filtered charts leaves us in doubt about the possible window for modeling (in case of the classical models), so we will keep 100 as the modeling window (nearly two weeks). This is a parameter that is worth empirically studying later on.

It is worth mentioning, that the `pacf` charts would definitely change drastically if we would use some differencing. Since down below we decide not to, we keep it as it is.

## Seasonal decomposition and the question of trends

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.tsatools import freq_to_period
import matplotlib.pyplot as plt

analysis = seasonal_decompose(df["pm2.5"], freq=freq_to_period(df.index.inferred_freq))

analysis.plot()
plt.show()

Well, the default setting (infer periods - hourly) is rather uninformative, so it is maybe worth using some domain knowledge here, and use yearly frequency.

In [ ]:
analysis = seasonal_decompose(df["pm2.5"], freq=24*365)

analysis.plot()
plt.show()

We do get the first impression, that there is no overarching simple trend, as well as there are non-trivial seasonal patterns. At a later stage we should investigate differencing regimes, but for now, we leave the data as is.

# Train, valid, test split - before normalization

Contamination by the normalization values is a distant possibility, but let's stick to paranoid practices.

In [ ]:
VALID_AND_TEST_SIZE=0.1

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_else, y_train, y_else = train_test_split(df, df["pm2.5"], test_size=VALID_AND_TEST_SIZE*2, shuffle=False)
X_valid, X_test, y_valid, y_test = train_test_split(X_else, y_else, test_size=0.5, shuffle=False)


We could have used `temporal_split` from `seglearn`, but that would have cast everything to numpy, so it was more convenient this way for now. Using `seglearn` is encouraged - if we would like to go into classical modeling.

# Data normalization

Our default assumption is to use Scikit's minmax scaler for easier learning by neural models.

But there are some exceptions:

## How to normalize dates?

For the year it is more tricky, it is basically an ordinal.
Subtracting the first year is nice, but how to handle the normalization to 0,1?

We could use 2018 as a max, but **WE WOULD HAVE TO WRITE A BIG CAVEAT MESSAGE FOR DEPLOY PEOPLE!**

So it should be something like  `(df.year - (df.year.min())-1)/((df.year.max()-df.year.min())*2)` (-1 is for avoiding zero, making the life of the network more easy...)

For now we stick to the minmax scaler (living risky... :-)

For month, day, hour default assumption is, scikit's minmax scaler could work, but we will choose a more elaborate solution from [here](https://ianlondon.github.io/blog/encoding-cyclical-features-24hour-time/). This capitalizes on the circular nature of these quasi ordinals.


In [ ]:
from sklearn.preprocessing import MinMaxScaler
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
# I literally hate when a standard Scikit function throws big bunches of warnings 
# - though suppressing them is a dangerous practice. Hence this comment. 

def minmax_scale(df_x,series_y, normalizers=None):
    features_to_minmax = ["year","pm2.5","DEWP","TEMP","PRES","Iws","Is","Ir"]

    if not normalizers:
        normalizers = {}

    for feat in features_to_minmax:
        if feat not in normalizers:
            normalizers[feat] = MinMaxScaler()
            normalizers[feat].fit(df_x[feat].values.reshape(-1, 1))
        
        df_x[feat] = normalizers[feat].transform(df_x[feat].values.reshape(-1, 1))

    series_y=normalizers["pm2.5"].transform(series_y.values.reshape(-1, 1))

    return df_x, series_y, normalizers

In [ ]:

X_train_norm, y_train_norm, normalizers = minmax_scale(X_train, y_train)
X_valid_norm, y_valid_norm, _ = minmax_scale(X_valid, y_valid, normalizers=normalizers)
X_test_norm, y_test_norm, _ = minmax_scale(X_test, y_test, normalizers=normalizers)


## Encoding of ordinals

The encoding of `cbwd` is interesting, since it is an ordinal again, or better to say not even that, it has a nice circular topology, so we will use the same sin-cos solution.

Problem is, that there is a valid "zero" value, marked "cv" in there. We are tempted to replace that with 0.

In [ ]:

def encode_cyclicals(df_x):
    #"month","day","hour", "cdbw", "dayofweek"
    
    DIRECTIONS = {"N":1.0,"NE":2.0, "E":3.0, "SE":4.0, "S":5.0, "SW":6.0, "W":7.0, "NW":8.0, "cv":np.nan}

    df_x['month_sin'] = np.sin(2*np.pi*df_x.month/12)
    df_x['month_cos'] = np.cos(2*np.pi*df_x.month/12)
    df_x.drop('month', axis=1, inplace=True)
    
    df_x['day_sin'] = np.sin(2*np.pi*df_x.day/31)
    df_x['day_cos'] = np.cos(2*np.pi*df_x.day/31)
    df_x.drop('day', axis=1, inplace=True)

    df_x['dayofweek_sin'] = np.sin(2*np.pi*df_x.dayofweek/7)
    df_x['dayofweek_cos'] = np.cos(2*np.pi*df_x.dayofweek/7)
    df_x.drop('dayofweek', axis=1, inplace=True)
    
    df_x['hour_sin'] = np.sin(2*np.pi*df_x.hour/24)
    df_x['hour_cos'] = np.cos(2*np.pi*df_x.hour/24)
    df_x.drop('hour', axis=1, inplace=True)
    
    df_x.replace({'cbwd': DIRECTIONS}, inplace=True)
    df_x['cbwd'] = df_x['cbwd'].astype(np.float64) 

    df_x['cbwd_sin'] = np.sin(2.0*np.pi*df_x.cbwd/8.0)
    df_x['cbwd_sin'].replace(np.nan, 0.0, inplace=True) #Let's handle the case with no wind specially
    df_x['cbwd_cos'] = np.cos(2.0*np.pi*df_x.cbwd/8.0)
    df_x['cbwd_cos'].replace(np.nan, 0.0, inplace=True) #Let's handle the case with no wind specially
    df_x.drop('cbwd', axis=1, inplace=True)
    
    return df_x

In [ ]:
X_train_norm = encode_cyclicals(X_train_norm)
X_valid_norm = encode_cyclicals(X_valid_norm)
X_test_norm = encode_cyclicals(X_test_norm)

In [ ]:
X_train_norm

In [ ]:
#Just in case to ensure we did everything right
assert all(x==np.float64 for x in list(X_train_norm.dtypes))

It would be worth checking with some assertions that the manual normalizers work well. Let's leave it to later work.

It is also worth noting, that the normalizers should be saved and used in production.

# Creating target (y) and "windows" (X) for modeling

By default we use the next 24 hour value of "pm2.5" for prediction, that is, I would like to predict what the pm2.5 will be like **at this hour 24 hours from now.**

We use the quite handy **seglearn** package for this.

Because of computational reasons, we **use the window of 100 hours** to predict. Classical models would have hard time to accommodate substantially (like 5-10x) context windows, LSTM-s would suffer from the challenge of long term memory. After a basic run of modeling the next big challenge would be to investigate PACF structure more and use eg. stateful LSTM modeling to try to accommodate the large "lookback".   

In [ ]:
TIME_WINDOW=100
FORECAST_DISTANCE=24

In [ ]:
from seglearn.transform import FeatureRep, SegmentXYForecast, last

segmenter = SegmentXYForecast(width=TIME_WINDOW, step=1, y_func=last, forecast=FORECAST_DISTANCE)

X_train_rolled, y_train_rolled,_=segmenter.fit_transform([X_train_norm.values],[y_train_norm.flatten()])

In [ ]:
X_train_rolled[:1]

# For non-sequence models 

We have to "flatten" the data to be able to use classical, non-sequence regression models from Scikit.

**We only need to do this for X, any transformation of y is unnecessary.**

In [ ]:
X_train_rolled.shape

In [ ]:
shape = X_train_rolled.shape
X_train_flattened = X_train_rolled.reshape(shape[0],shape[1]*shape[2])
X_train_flattened.shape

In [ ]:
X_valid_rolled, y_valid_rolled,_=segmenter.fit_transform([X_valid_norm.values],[y_valid_norm.flatten()])

shape = X_valid_rolled.shape
X_valid_flattened = X_valid_rolled.reshape(shape[0],shape[1]*shape[2])

# Evaluation helper

Use this function to evaluate your models **on validation data.**

This assumes that your model has the `predict()` function, which is true for **Scikit-learn, XGBoost** and **Keras**, so you can can hand over any of those. 

A special issue by models optimized by iterative methods is to **get the final model**. **Early stopping and / or model save and reload** can help there.  

**WARNING: This is just a basic evaluation scheme, more thorough investigation needed in the future!**

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def evaluate_model(model, X_valid, y_valid_true):
    predictions = model.predict(X_valid)
    rms = sqrt(mean_squared_error(y_valid_true, predictions))
    print("Root mean squared error on valid:",rms)
    normalized_rms = normalizers["pm2.5"].inverse_transform(np.array([rms]).reshape(1, -1))[0][0]
    print("Root mean squared error on valid inverse transformed from normalization:",normalized_rms)
    return normalized_rms

# Classical modeling

In "classical" modeling we assume a multiple regression case, so we **DO NOT USE time series as such, but the "flat" versions of the data** as input. Output is the same. 

## Baseline - DummyPredictor

**TASK Create a dummy predictor as a baseline. Use Scikit-learn's builtin capability to do dummy models in regression case. Use the default setting, that is the prediction of the mean value.**

In [ ]:
from sklearn...

dummy_model = ...

dummy_model.fit(...)

### Evaluation

In [ ]:
result = evaluate_model(...)

## Fitting a RandomForest on raw data

**TASK: Fit a RandomForest from Scikit. Please be aware, that the number of trees in the model is having a strong influence on training time.** 

**Suggestion:** use couple of tens of trees, definitely << 100 to be able to wait it out...

**Pro tip:** To utilize all the CPU cores, use the right setting of n_jobs. That speeds things up.

In [ ]:
from sklearn...

N_ESTIMATORS = ???
RANDOM_STATE = 452543634

In [ ]:
RF_base_model = ...

RF_base_model.fit(...)

### Evaluation


In [ ]:
result = evaluate_model(...)

## Fitting a RandomForest on feature transformed data

**TASK: Since we use `seglearn`, we can try to capitalize on it's functionality to calculate features from the time time series. Use `FeatureRep` from `seglearn` to transform features, fit a RandomForest and hope for the best!**

In [ ]:
RF_feature_model = ...

feature_converter = ...

RF_feature_model.fit(feature_converter...)

### Evaluation

WARNING: This is just a basic evaluation scheme, more thorough investigation needed!

In [ ]:
result = evaluate_model(...,feature_converter..., ...)

## XGBoost for speedup

Use the XGBoost library to fit gradient boosted trees to the problem. They are usually way quicker to learn and many times at least on par with RandomForests, or better. Let's see!

In [ ]:
import xgboost as xgb
# If in trouble, use !pip install xgboost

# XGBoost needs it's custom data format to run quickly
dmatrix_train = xgb.DMatrix(data=X_train_flattened,label=y_train_rolled)
dmatrix_valid = xgb.DMatrix(data=X_valid_flattened,label=y_valid_rolled)

In [ ]:
params = {'objective': 'reg:linear', 'eval_metric': 'rmse', 'n_estimators': ??}

evallist = [(dmatrix_valid, 'eval'), (dmatrix_train, 'train')]

num_round = ?? #Can easily overfit, experiment with it!

xg_reg = xgb.train(...)

In [ ]:
result = evaluate_model(...,dmatrix...,...)

# Building an LSTM model

## Modeling assumptions

**TASK:** We believe, that the time dependent structure of this dataset is complex, so we try to use LSTM models from Keras. We are not explicitly utilizing **statefulness**, that is a **major area to be investigated later on**.

More information on statefulness can be found [here](https://philipperemy.github.io/keras-stateful-lstm/).


Fit an LSTM model **on the time series - non-flat - data!**.

Use:
1. At least 1 LSTM layer
2. A dense layer for output - think about activation! This is a regression case!

**Very advisable** - but optional - to use Dropout. You can not use it everywhere, though... Experiment!

You are allowed to use functional API, but for this **Sequential API is sufficient.**

You **can use LeraningRateScheduler** if you like.

In [ ]:
LSTM_CELL_SIZE=??
BATCH_SIZE = ??
EPOCHS = ??
DROPOUT_RATE=??

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras import backend as be
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler

column_count=len(X_train_???.columns) #Remember,column count before rolling...

be.clear_session()

# You might very well be needing it!
# Remeber to save only what is worth it from validation perspective...
# model_saver = ModelCheckpoint(...)

# If you need it...
#def schedule(epoch, lr):
#    ...
#    return lr

#lr_scheduler = LearningRateScheduler(schedule)

# Build your whole LSTM model here!
model = ...

#For shape remeber, we have a variable defining the "window" and the features in the window...

model.compile(loss='mean_squared_error', optimizer=???)
# Fit on the train data
# USE the batch size parameter!
# Use validation data - warning, a tuple of stuff!
# Epochs as deemed necessary...
# You should avoid shuffling the data maybe.
# You can use the callbacks for LR schedule or model saving as seems fit.
history = model.fit(...)

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
# You can use the early stopped model OR load it. 
# For that you have to import the load function...
# IF AND ONLY IF loading, it is good practice to throw out the trash from the graph...
# be.clear_session()


result = evaluate_model(...)

In [ ]:
assert result < 86.0

# Things that should be improved

- More conclusive investigation of PACF for better time window estimate
    - It can well be, that long windows do not add that much to the performance
- More interesting features for XGBoost (like from [tsfresh](https://tsfresh.readthedocs.io/en/latest/)), since present features are a disaster
- MOST IMPORTANT: **More thorough error / prediction analysis!!!**
- LSTM with **Custom iterator with stateful model**
- Investigation of different loss function (eg. MAE) for training. (And with it, think abut the importance of extreme values: do we think they are outliers? Are they interesing to predict?)
- Investigation of "teacher forcing" for LSTM-s in Keras (if it makes sense)

# Conclusion

Even with decent amount of struggle, the "dummy" of always using the mean is very appealing, so it seems, this is not that easy of a task 24 hours in advance. Further investigation of classical as well as neural models remains open!


# Final test

We did not use the final test, since our investigations are not concluded yet. Remember: using it once before project "go live" is a good practice!